In [3]:
import os
import json


In [25]:
SOF_VIEW_OUT_DIR="../src/sof"

In [22]:
class ViewDef:
    def __init__(self, name, resource, select, where = None):
        self.name = name
        self.resource = resource
        self.select = select
        self.where = where
    def to_json(self):
        return json.dumps(self.to_dict(), indent=2)
    def to_dict(self):
        return { k:v for (k,v) in self.__dict__.items() if v}
    
    def save_to(self, out_dir):
        with open(f"{out_dir}/{self.name}.json", 'w') as f:
            f.write(self.to_json())
    

### Patient

In [38]:
rv_patient = ViewDef('rv_patient', 'Patient', 
    select = [
        {
            "column": [
                {
                    "name": "patient_id",
                    "path": "getResourceKey()",
                    "collection": False
                },
                {
                    "name": "gender",
                    "path": "gender",
                    "collection": False
                },
                {
                    "name": "race_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "race_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
                {
                    "name": "ethnicity_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "ethnicity_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
            ]
        }
    ]
)
rv_patient.save_to(SOF_VIEW_OUT_DIR)

### Encounter(ICU) 


In [27]:
rv_icu_encounter = ViewDef('rv_icu_encounter', 'Encounter',
                    select = [
                        {
                            "column": [
                                {
                                    "name": "encounter_id",
                                    "path": "getResourceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "patient_id",
                                    "path": "subject.getReferenceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "admit_date",
                                    "path": "period.start",
                                    "type": "dateTime",
                                    "collection": False
                                },
                                {
                                    "name": "disch_date",
                                    "path": "period.end",
                                    "type": "dateTime",
                                    "collection": False
                                }

                            ]
                        }
                    ],
                    where= [
                        {
                            "path":"class.system = 'http://terminology.hl7.org/CodeSystem/v3-ActCode'"
                        },
                        {
                            "path":"class.code = 'ACUTE'"
                        },

                    ]
                    )

rv_icu_encounter.save_to(SOF_VIEW_OUT_DIR)

### ICU Stay Detail

## Oxygen Delivery Device

In [28]:
CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'

def obs_str_for_coding(system, code, view_name):
    return ViewDef(view_name, 'Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "charttime",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "storetime",
                                         "path": "issued",
                                         "type": "instant",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueString",
                                         "type": "string",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                        )

In [30]:
rv_oxygen_delivery_device = obs_str_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, '226732', 'rv_o2_delivery_device')
rv_oxygen_delivery_device.save_to(SOF_VIEW_OUT_DIR)

In [14]:
# # Alternative way to get the first intervention
# stays_with_interventions = spark.sql("""
# WITH odd AS (
#     SELECT subject_id, stay_id, charttime, o2_delivery_device AS o2_delivery_device FROM dv_oxygen_delivery_device
# ), odd1 AS (SELECT *, 
#     CASE
#     -- tracheostomy
#     WHEN o2_delivery_device IN (
#         'Tracheostomy tube',
#         'Trach mask ' -- 16435 observations
#         -- 'T-piece', -- 1135 observations (T-piece could be either InvasiveVent or Tracheostomy)
#         ) THEN 'Tracheostomy'
#     -- mechanical / invasive ventilation
#     WHEN o2_delivery_device IN (
#         'Endotracheal tube'
#         ) THEN 'InvasiveVent'
#     -- NIV
#     WHEN o2_delivery_device IN (
#         'Bipap mask ', -- 8997 observations
#         'CPAP mask ' -- 5568 observations
#         ) THEN 'NonInvasiveVent'
#     -- high flow nasal cannula
#     when o2_delivery_device IN (
#         'High flow nasal cannula' -- 925 observations
#         ) THEN 'HFNC'
#     -- non rebreather
#     WHEN o2_delivery_device IN ( 
#         'Non-rebreather', -- 5182 observations
#         'Face tent', -- 24601 observations
#         'Aerosol-cool', -- 24560 observations
#         'Venti mask ', -- 1947 observations
#         'Medium conc mask ', -- 1888 observations
#         'Ultrasonic neb', -- 9 observations
#         'Vapomist', -- 3 observations
#         'Oxymizer', -- 1301 observations
#         'High flow neb', -- 10785 observations
#         'Nasal cannula'
#         ) THEN 'SupplementalOxygen'
#     WHEN o2_delivery_device in (
#         'None'
#      ) THEN 'None'
#     -- not categorized: other
#     ELSE NULL END AS ventilation_status
# FROM odd WHERE o2_delivery_device  IS NOT NULL
# ), ventilation_type AS (
# SELECT subject_id, stay_id, charttime AS starttime, ventilation_status FROM odd1
# ), iws AS (
#     SELECT stay_id, starttime as  inttime, ventilation_status AS int_type,
# row_number() OVER (PARTITION BY stay_id ORDER BY starttime) AS int_sequence
# FROM ventilation_type
# WHERE ventilation_status NOT in ('None', 'SupplementalOxygen') AND ventilation_status IS NOT NULL
# )
# SELECT stay_id, inttime, int_type FROM iws WHERE int_sequence = 1
# """)
# stays_with_interventions.count()

33376

## Observations 
### Chartevents

In [31]:
def obs_for_coding(system, code, view_name):
    return ViewDef(view_name, 'Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "date",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                         )

CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'

def obs_chartevent_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, code, view_name)

def obs_labitems_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_D_LABITEMS, code, view_name)

#O2 Flow

In [37]:
rv_obs_o2_flow = ViewDef('rv_obs_o2_flow', 'Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "charttime",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "storetime",
                                         "path": "issued",
                                         "type": "instant",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     },
                                     {
                                        "name": "code",
                                        "path": "code.coding.code",
                                        "collection": False,
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{CS_MIMIC_CHARTEVENTS_D_ITEMS}'"
                             },
                             {
                                 "path":f"code.coding.code = '223834' or code.coding.code = '227582' or code.coding.code = '227287'"
                             },

                         ]
)
rv_obs_o2_flow.save_to(SOF_VIEW_OUT_DIR)

#CHARTEVENTS

In [32]:
obs_resp_rate = obs_chartevent_for_code('224690', 'rv_obs_resp_rate')
obs_resp_rate.save_to(SOF_VIEW_OUT_DIR)

In [17]:
obs_spo2 = obs_chartevent_for_code('220277', 'obs_spo2')
obs_spo2.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|33fbe9fd-9006-56d...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T03:00:...|100.000000|   %|
|cdc16b23-7699-531...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T02:00:...|100.000000|   %|
|673ffabc-6c96-554...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T01:00:...|100.000000|   %|
|65786cc2-39ff-543...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T23:00:...|100.000000|   %|
|b6aca445-3efd-570...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T22:00:...|100.000000|   %|
|e5cff1d5-66e5-505...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T21:00:...|100.000000|   %|
|1dd13aeb-366c-588...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T19:00:.

In [18]:
obs_heart_rate = obs_chartevent_for_code('220045', 'obs_heart_rate')
obs_heart_rate.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|2baf4fb9-6887-532...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T05:00:...| 80.000000| bpm|
|c113df93-2362-59f...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T04:00:...| 80.000000| bpm|
|b30e5e3f-382a-576...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T03:00:...| 78.000000| bpm|
|f2e1b838-abf5-57b...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T02:00:...| 80.000000| bpm|
|328010f0-f325-533...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T01:00:...| 68.000000| bpm|
|78b84b7e-4530-509...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T00:00:...| 75.000000| bpm|
|07acc617-0e3f-565...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T23:00:.

In [19]:
#
# alike mimiciv_derrived.vital_signs
#
derr_vital_signs_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS resp_rate, NULL AS heart_rate, NULL AS spo2 FROM obs_resp_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, value AS heart_rate,  NULL AS spo2 FROM obs_heart_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, NULL AS heart_rate,  value AS spo2 FROM obs_spo2 
) 
SELECT patient_id, encounter_id, obs_time, FIRST(resp_rate) AS resp_rate, FIRST(heart_rate) AS heart_rate, FIRST(spo2) AS spo2 FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_vital_signs_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,resp_rate,heart_rate,spo2
0,Patient/981c9fae-f498-58fc-9c9d-e400dae485ff,Encounter/a92486e2-c0ca-58ac-a2e6-ddb2ef676aba,2185-04-15 20:00:00,28.000000,None,None
1,Patient/ee14b6e8-9a6d-5484-84d8-dee2bc795128,Encounter/74f701cd-33a8-5862-97e6-6f5c0e2a4d13,2181-11-23 11:00:00,18.000000,None,None
2,Patient/ee1dd889-5efd-5297-a3b5-19980ea145ab,Encounter/85edc39a-ef60-55aa-9cbd-82d672d94e79,2163-10-14 05:00:00,19.000000,None,None
3,Patient/dc68774d-2198-5caa-8dc5-769bb7afd090,Encounter/e4265880-8e09-5500-a3b8-ae13fbeb4426,2168-02-06 10:00:00,16.000000,None,None
4,Patient/a6e2173f-d1ef-55d1-999e-4da6be7b5f1f,Encounter/41b20ce6-142c-5565-bae3-80710b284504,2146-01-06 19:00:00,13.000000,None,None
...,...,...,...,...,...,...
138458,Patient/f642662e-6db3-56db-815c-59afc9adbeba,Encounter/9315c359-300f-5afd-bc24-696b46c2f51f,2158-11-29 01:01:00,None,None,92.000000
138459,Patient/774816bf-2874-5a1a-a342-9c1dd993ce78,Encounter/0e0cd3f5-bd08-5a7a-8ed5-cf4482d37234,2119-10-23 05:25:00,None,None,94.000000
138460,Patient/240a5e20-49fb-5924-bc9d-020bc2331f21,Encounter/1cad8c5f-0be4-5e00-a8d4-4dba8bbbf0ad,2181-04-09 05:25:00,None,None,97.000000
138461,Patient/2402a0f7-88d3-5978-8296-d01283cec2df,Encounter/2d6f5df8-6847-5291-9766-e74ecb97314c,2189-04-23 04:04:00,None,None,95.000000


### Labevents

In [20]:
# bg so2
obs_bg_so2 = obs_labitems_for_code('50817', 'obs_bg_so2')
# AND < 100
obs_bg_so2.show()

+--------------------+--------------------+--------------------+--------------------+---------+----+
|                  id|          patient_id|        encounter_id|                date|    value|unit|
+--------------------+--------------------+--------------------+--------------------+---------+----+
|a13ebf5d-d467-5a8...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T08:44:...|64.000000|   %|
|50cf9af5-fe15-58c...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T04:33:...|66.000000|   %|
|9dbc3b8c-ba80-542...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T15:07:...|67.000000|   %|
|bab53891-1a57-504...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T08:24:...|95.000000|   %|
|98be9865-9de2-56d...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T08:27:...|77.000000|   %|
|46d1f001-5591-566...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T23:56:...|76.000000|   %|
|3f44fbc5-8a05-569...|Patient/b37d3c55-...|                NULL|2142-09-27T09:22:...|20.000

In [21]:
# bg so2
obs_bg_pco2 = obs_labitems_for_code('50818', 'obs_bg_pco2')
obs_bg_pco2.show()

+--------------------+--------------------+--------------------+--------------------+---------+-----+
|                  id|          patient_id|        encounter_id|                date|    value| unit|
+--------------------+--------------------+--------------------+--------------------+---------+-----+
|4f59abea-cfc1-52a...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T10:49:...|29.000000|mm Hg|
|232aa8a8-e9cc-502...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-30T04:12:...|30.000000|mm Hg|
|691b5fab-8c5b-529...|Patient/b37bfbf8-...|                NULL|2167-11-28T01:35:...|32.000000|mm Hg|
|384a0bbf-6e22-5a7...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T08:41:...|30.000000|mm Hg|
|7e18dbaf-4d7f-53b...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T04:29:...|30.000000|mm Hg|
|2d6a552f-8c7f-581...|Patient/b37bfbf8-...|Encounter/0130857...|2167-12-01T05:12:...|31.000000|mm Hg|
|24e34308-14a1-573...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T01:45:.

In [22]:
# hemoglobin
obs_bg_hemoglobin = obs_labitems_for_code('50811', 'obs_bg_hemoglobin')
obs_bg_hemoglobin.toPandas()

,id,patient_id,encounter_id,date,value,unit
0,893744a0-2950-54e1-a5e3-9e6cd0e5a585,Patient/b37b2674-06d0-5bb7-8da0-0ece76cee89a,None,2168-02-17T12:00:00-05:00,16.600000,g/dL
1,b3981c0c-e358-5f4b-95a7-6b43f9f3049f,Patient/b37dc22c-5665-5df9-bd04-42839c5f71ae,Encounter/f66b075c-f4e4-5144-bc11-bb49e70d80a9,2182-01-17T18:27:00-05:00,10.300000,g/dL
2,9268948a-379d-54ce-98f1-1ec52dd474aa,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T10:33:00-05:00,12.300000,g/dL
3,9afe3ec7-debb-501a-a31b-a727c965334e,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T12:37:00-05:00,11.300000,g/dL
4,de042f86-647f-5342-804b-2f38b67d7d74,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T15:18:00-05:00,11.100000,g/dL
...,...,...,...,...,...,...
111628,2fff3542-792b-5260-b07c-6f310e942d23,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T12:10:00-04:00,6.800000,g/dL
111629,d7a374d3-a1ab-53e3-a54e-93267e0baf1e,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T12:48:00-04:00,6.600000,g/dL
111630,abee4d7b-285b-5a66-850b-d326e6ba47f3,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T13:03:00-04:00,6.600000,g/dL
111631,1158866f-9ce5-5181-88f5-dcd791691cbb,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T14:19:00-04:00,7.800000,g/dL


In [23]:
#
# alike mimiciv_derrived.bg
#
derr_bg_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS so2, NULL AS pco2, NULL AS hemoglobin  FROM obs_bg_so2 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, value AS pco2, NULL AS hemoglobin FROM obs_bg_pco2
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, NULL AS pco2, value AS hemoglobin FROM obs_bg_hemoglobin  
) 
SELECT patient_id, encounter_id, obs_time, FIRST(so2) AS so2, FIRST(pco2) AS pco2, FIRST(hemoglobin) AS hemoglobin FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_bg_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,so2,pco2,hemoglobin
0,Patient/0b330ffd-8b21-51d0-b91f-9f39de481ad2,Encounter/f325ab8a-a604-585b-89d8-e5d2676c0221,2114-08-01 20:04:00,65.000000,None,None
1,Patient/29abe168-5a11-512e-b5cb-7b686188a350,Encounter/f0f6f2ab-0602-5446-9188-30d7be2d9a45,2114-06-14 01:22:00,98.000000,None,None
2,Patient/f24e8cf5-d268-5c5c-be0c-ca645224f8f6,Encounter/1ba38cb1-4d5f-5986-89df-2a3820e7b634,2151-06-07 18:01:00,98.000000,None,None
3,Patient/a15664ba-9f04-5223-ad3e-33c968abf70b,Encounter/42252eb0-d954-5202-b3d1-0b11c9158d0a,2145-06-20 12:33:00,50.000000,None,None
4,Patient/d10c8c20-3658-53c6-8594-a2e1afafaa3e,Encounter/4e53cf64-63f7-51f7-b644-c5fefc89d3d2,2148-10-10 22:17:00,93.000000,None,None
...,...,...,...,...,...,...
11252,Patient/d9f25663-540e-5db7-97be-9782741944fe,None,2114-05-15 14:24:00,None,None,8.500000
11253,Patient/850346fa-4499-5c93-846c-b628e0d80367,Encounter/03f35944-3982-59e2-a898-9735dab87ed0,2147-11-09 12:33:00,None,None,8.100000
11254,Patient/1efcb51a-2f45-5622-a2ec-b11714ef9591,Encounter/c1ef6e15-f593-515c-b923-ce28d093a25a,2185-12-30 16:07:00,None,None,7.400000
11255,Patient/5b43140c-d1b3-552a-a5e2-422e434ac595,None,2190-11-22 11:50:00,None,None,14.600000


## Encounters (ICU) 